# Reordering



In [1]:
import numpy as np
from scipy.sparse import diags, eye, kron

def tridiag(a, b, c, n):
    # Create a tridiagonal matrix
    return diags([a*np.ones(n-1), b*np.ones(n), c*np.ones(n-1)], [-1, 0, 1], format='csr')

def fd3d(nx, ny, nz, alpx, alpy, alpz, dshift):
    # Create tridiagonal matrices tx, ty, tz
    tx = tridiag(-1+alpx, 2, -1-alpx, nx)
    ty = tridiag(-1+alpy, 2, -1-alpy, ny)
    tz = tridiag(-1+alpz, 2, -1-alpz, nz)

    # Compute A using Kronecker products
    A = kron(eye(ny), tx) + kron(ty, eye(nx))
    if nz > 1:
        A = kron(eye(nz), A) + kron(tz, eye(nx*ny))

    # Subtract dshift times identity matrix from A
    A -= dshift * eye(nx*ny*nz)

    return A

In [ ]:
A = fd3d(15,1,1,0,0,0,0)

In [ ]:
from scipy.sparse.linalg import splu
lu = splu(A,'NATURAL')
print(lu.nnz)


In [ ]:
lu2 = splu(A,'COLAMD')
print(lu2.nnz)

In [ ]:
import matplotlib.pyplot as plt
plt.spy(lu.L+lu.U)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.spy(lu2.L+lu2.U)
plt.show()

In [ ]:
!sudo apt-get install -y metis

In [ ]:
pip install metis

In [5]:
import numpy as np
from scipy.sparse import diags, eye, kron

def tridiag(a, b, c, n):
    # Create a tridiagonal matrix
    return diags([a*np.ones(n-1), b*np.ones(n), c*np.ones(n-1)], [-1, 0, 1], format='csr')

def fd3d(nx, ny, nz, alpx, alpy, alpz, dshift):
    # Create tridiagonal matrices tx, ty, tz
    tx = tridiag(-1+alpx, 2, -1-alpx, nx)
    ty = tridiag(-1+alpy, 2, -1-alpy, ny)
    tz = tridiag(-1+alpz, 2, -1-alpz, nz)

    # Compute A using Kronecker products
    A = tx
    if ny > 1:
        A = kron(eye(ny), A) + kron(ty, eye(nx))
        if nz > 1:
            A = kron(eye(nz), A) + kron(tz, eye(nx*ny))

    # Subtract dshift times identity matrix from A
    A -= dshift * eye(nx*ny*nz)

    return A

In [6]:
nnx = 6
nny = 1
nnz = 1

In [7]:
A = fd3d(nnx,nny,1,0,0,0,0)

In [ ]:
import scipy as sp
import metis

In [8]:
print(A)

  (0, 0)	2.0
  (0, 1)	-1.0
  (1, 0)	-1.0
  (1, 1)	2.0
  (1, 2)	-1.0
  (2, 1)	-1.0
  (2, 2)	2.0
  (2, 3)	-1.0
  (3, 2)	-1.0
  (3, 3)	2.0
  (3, 4)	-1.0
  (4, 3)	-1.0
  (4, 4)	2.0
  (4, 5)	-1.0
  (5, 4)	-1.0
  (5, 5)	2.0


In [ ]:
def sparse_matrix_to_adjacency_list(sparse_matrix):
    adjacency_list = {}
    rows, cols = sparse_matrix.nonzero()

    for row, col in zip(rows, cols):
        if row not in adjacency_list:
            adjacency_list[row] = []
        adjacency_list[row].append(col)

    return adjacency_list

In [ ]:
import metis
import networkx as nx
import matplotlib.pyplot as plt

# Convert the sparse matrix to an adjacency list
adjlist = sparse_matrix_to_adjacency_list(A)
G = nx.Graph()

# Add edges to the graph
for node, neighbors in adjlist.items():
    for neighbor in neighbors:
        G.add_edge(node, neighbor)

# Apply nested dissection ordering
_, parts = metis.part_graph(G, 2)
print("Nested Dissection Ordering:", parts)

# Assign colors to nodes based on their partition
colors = ['red' if part == 0 else 'blue' for part in parts]


pos = {node: (node % grid_width, -node // grid_width) for node in G.nodes()}

# Draw the graph
pos = nx.spring_layout(G)  # positions for all nodes
nx.draw(G, pos, node_color=colors, with_labels=True, node_size=700)
plt.show()


In [ ]:
import networkx as nx
from scipy.sparse import csr_matrix
import numpy as np
import metis

def sparse_matrix_to_graph(sparse_matrix):
    """Convert a scipy sparse matrix to a networkx graph."""
    G = nx.Graph()
    G.add_nodes_from(range(sparse_matrix.shape[0]))
    cx = sparse_matrix.tocoo()
    for i, j, v in zip(cx.row, cx.col, cx.data):
        # Add each edge once
        if i <= j:
            G.add_edge(i, j, weight=v)
    return G

def find_separator(G):
    """Partition the graph and identify the separator nodes, ensuring separator nodes are not in part_1 or part_2."""
    # Partition the graph into two parts
    edgecuts, parts = metis.part_graph(G, 2)

    # Initially identify nodes in each part
    initial_part_1 = [node for node in G.nodes() if parts[node] == 0]
    initial_part_2 = [node for node in G.nodes() if parts[node] == 1]

    # Identify separator nodes: nodes with edges that cross the partition
    separator = set()
    for edge in G.edges():
        if parts[edge[0]] != parts[edge[1]]:
            separator.add(edge[0])
            separator.add(edge[1])

    # Remove separator nodes from initial_part_1 and initial_part_2
    part_1 = [node for node in initial_part_1 if node not in separator]
    part_2 = [node for node in initial_part_2 if node not in separator]

    return part_1, part_2, list(separator)



# Convert the sparse matrix to a graph
G2 = sparse_matrix_to_graph(A)

# Find the separator
part_1, part_2, separator = find_separator(G2)
print("Part 1:", part_1)
print("Part 2:", part_2)
print("Separator:", separator)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Assuming G2 is your graph, and part_1, part_2, and separator lists are defined

# Grid size
grid_width, grid_height = nnx, nny

# Generate grid positions for a nnx*nny grid
pos = {node: (node % grid_width, -node // grid_width) for node in G2.nodes()}

# Assign colors and sizes based on node roles: part_1, part_2, and separator
colors = ['red' if node in part_1 else 'blue' if node in part_2 else 'green' for node in G2.nodes()]
sizes = [700 if node not in separator else 1000 for node in G2.nodes()]  # Larger size for separator nodes

# Draw the graph maintaining the grid topology
nx.draw(G2, pos, node_color=colors, with_labels=True, node_size=sizes, edge_color='gray')

plt.show()